## 결과


> #1 Age, Cabin, Ticket 제거 valid 0.819 lb 0.747

> #2 Cabin, Ticket 제거, Age 양자화 후 보간 valid 0.941 lb 0.751

> #3 Cabin, Ticket 제거, Age 평균 보간 : valid 0.866 lb 0.746

> #4 Age, Ticket 제거, Cabin 카테고리 분류 : valid 0.819 lb 0.762

> #5 Cabin, Ticket 제거, Age 선형회귀 : valid lb 0.866 lb 0.746

## 코드

In [1]:
import pandas as pd
import numpy as np

In [2]:
from catboost import CatBoostClassifier, CatBoostRegressor
import catboost as cb

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

In [22]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submit = pd.read_csv('submission.csv')

train['Embarked'] = train['Embarked'].fillna('S')
test.loc[test[test.Fare.isnull()].index, 'Fare'] = 8.0500

train['Sex'] = train['Sex'].replace('male', 0)
train['Sex'] = train['Sex'].replace('female', 1)
train['Embarked'] = train['Embarked'].replace('C', 0)
train['Embarked'] = train['Embarked'].replace('Q', 1)
train['Embarked'] = train['Embarked'].replace('S', 2)
train.drop(columns=['PassengerId', 'Name'], inplace=True)

test['Sex'] = test['Sex'].replace('male',0)
test['Sex'] = test['Sex'].replace('female',1)
test['Embarked'] = test['Embarked'].replace('C', 0)
test['Embarked'] = test['Embarked'].replace('Q', 1)
test['Embarked'] = test['Embarked'].replace('S', 2)
test.drop(columns=['PassengerId', 'Name'], inplace=True)

## #1 Age, Cabin, Ticket 제거

In [23]:
_train, valid = train_test_split(train, test_size = 0.1, random_state=42)

train_x, train_y = _train.drop(columns=['Survived']), _train['Survived']
valid_x, valid_y = valid.drop(columns=['Survived']), valid['Survived']
_test = test.copy()

for data in [train_x, valid_x, _test]:
    data.drop(columns=['Age', 'Cabin', 'Ticket'], inplace=True)

train_dataset = cb.Pool(train_x, train_y) 
valid_dataset = cb.Pool(valid_x, valid_y)
test_dataset = cb.Pool(_test)

model = cb.CatBoostClassifier(loss_function='Logloss', silent=True, random_state=42)
model.fit(train_dataset)

pred = model.predict(valid_dataset)
score = roc_auc_score(valid_y.values, pred)
print(score)

submit['Survived'] = model.predict(test_dataset)
submit.to_csv('#1.csv', index=False)

0.8194444444444445


## #2 Cabin, Ticket 제거, Age 양자화 후 보간

In [21]:
_train = train.copy()

best_score = 0
related_cols = ['Pclass', 'SibSp']
for board in range(1, len(related_cols)+1):
    cols = related_cols[:board]
    print(cols)
    
    for qnt in range(2, 11):
        _train.Age = train.Age.copy()
        _train.Age = pd.cut(train.Age, qnt, labels=range(qnt))
        table = _train[cols + ['Age']].value_counts().sort_index()
        
        for index, *value in _train.loc[_train.Age.isnull(), cols].reset_index().values:
            for i in range(len(cols)):
                if (table.index.get_level_values(i) == value[i]).any():
                    continue
                while not (table.index.get_level_values(i) == value[i]).any():
                    value[i] -= 1
                break
            
            value = tuple(value)
            age = table.loc[value].sort_values(ascending=False).index[0]
            _train.loc[index, 'Age'] = age
        
        _train.Age = _train.Age.astype(int)
        
        _train, valid = train_test_split(_train, test_size = 0.1, random_state=42)
        
        train_x, train_y = _train.drop(columns=['Survived']), _train['Survived']
        valid_x, valid_y = valid.drop(columns=['Survived']), valid['Survived']
        _test = test.copy()

        for data in [train_x, valid_x,]:
            data.drop(columns=['Cabin', 'Ticket'], inplace=True)

        train_dataset = cb.Pool(train_x, train_y) 
        valid_dataset = cb.Pool(valid_x, valid_y)

        model = cb.CatBoostClassifier(loss_function='Logloss', silent=True, random_state=42)
        model.fit(train_dataset)

        pred = model.predict(valid_dataset)
        score = roc_auc_score(valid_y.values, pred)
        print(qnt, score)
        
        if best_score < score:
            best_comb = (cols, qnt)
            best_score = score
            
            _test = test.copy()
            _test.Age = pd.cut(test.Age, qnt, labels=range(qnt))

            for index, *value in _test.loc[_test.Age.isnull(), cols].reset_index().values:
                for i in range(len(cols)):
                    if (table.index.get_level_values(i) == value[i]).any():
                        continue
                    while not (table.index.get_level_values(i) == value[i]).any():
                        value[i] -= 1
                    break

                value = tuple(value)
                age = table.loc[value].sort_values(ascending=False).index[0]
                _test.loc[index, 'Age'] = age
                
            _test.Age = _test.Age.astype(int)
            
            _test.drop(columns=['Cabin', 'Ticket'], inplace=True)
            test_dataset = cb.Pool(_test)
            
            submit['Survived'] = model.predict(test_dataset)
            submit.to_csv(f'#2.csv', index=False)
            
print("best is ", best_comb)

['Pclass']
2 0.8657407407407408
3 0.8068181818181819
4 0.6259259259259259
5 0.8003246753246754
6 0.8761574074074073
7 0.9011824324324326
8 0.8430127041742287
9 0.8506944444444443
10 0.6676923076923077
['Pclass', 'SibSp']
2 0.9388111888111889
3 0.9411764705882353
4 0.820855614973262
5 0.8947368421052632
6 0.5892857142857143
7 0.6458333333333333
8 0.75
9 0.8785714285714286
10 0.44999999999999996
best is  (['Pclass', 'SibSp'], 3)


## #3 Cabin, Ticket 제거, Age 평균 보간

In [12]:
_train, valid = train_test_split(train, test_size = 0.1, random_state=42)
_test = test.copy()

table = _train.pivot_table(values = 'Age', index = 'Pclass', aggfunc = 'mean')
for class_num in [1, 2, 3]:
    _train.loc[_train.loc[_train.Age.isnull()].loc[_train.Pclass == class_num].index]['Age'] = table.loc[class_num]
    valid.loc[valid.loc[valid.Age.isnull()].loc[valid.Pclass == class_num].index]['Age'] = table.loc[class_num]
    _test.loc[_test.loc[_test.Age.isnull()].loc[_test.Pclass == class_num].index]['Age'] = table.loc[class_num]

train_x, train_y = _train.drop(columns=['Survived']), _train['Survived']
valid_x, valid_y = valid.drop(columns=['Survived']), valid['Survived']

for data in [train_x, valid_x, _test]:
    data.drop(columns=['Cabin', 'Ticket'], inplace=True)

train_dataset = cb.Pool(train_x, train_y) 
valid_dataset = cb.Pool(valid_x, valid_y)
test_dataset = cb.Pool(_test)

model = cb.CatBoostClassifier(loss_function='Logloss', silent=True, random_state=42)
model.fit(train_dataset)

pred = model.predict(valid_dataset)
score = roc_auc_score(valid_y.values, pred)
print(score)

submit['Survived'] = model.predict(test_dataset)
submit.to_csv('#3.csv', index=False)

0.8657407407407408


## #4 Age, Ticket 제거, Cabin 카테고리 분류

In [20]:
_train = train.copy()
_train['Cabin'] = train['Cabin'].astype('category').cat.codes
_test = test.copy()
_test['Cabin'] = test['Cabin'].astype('category').cat.codes

cabin_train, cabin_blank = _train[_train.Cabin != -1], _train[_train.Cabin == -1]
cabin_dataset = cb.Pool(cabin_train.drop(columns=['Survived', 'Cabin', 'Age', 'Ticket']), cabin_train['Cabin'])

model = cb.CatBoostClassifier(loss_function='MultiClass', silent=True, random_state=42)
model.fit(cabin_dataset)

pred = model.predict(cabin_blank.drop(columns=['Survived', 'Cabin', 'Age', 'Ticket']))
_train.loc[cabin_blank.index]['Cabin'] = np.reshape(pred, (-1,))

_, cabin_test = _test[_test.Cabin != -1], _test[_test.Cabin == -1]
cabin_dataset = cb.Pool(cabin_test.drop(columns=['Cabin', 'Age', 'Ticket']), cabin_test['Cabin'])

pred = model.predict(cabin_dataset)
_test.loc[cabin_test.index]['Cabin'] = np.reshape(pred, (-1,))

_train, valid = train_test_split(_train, test_size = 0.1, random_state=42)

train_x, train_y = _train.drop(columns=['Survived']), _train['Survived']
valid_x, valid_y = valid.drop(columns=['Survived']), valid['Survived']

for data in [train_x, valid_x, _test]:
    data.drop(columns=['Age', 'Ticket'], inplace=True)

train_dataset = cb.Pool(train_x, train_y) 
valid_dataset = cb.Pool(valid_x, valid_y)
test_dataset = cb.Pool(_test)

model = cb.CatBoostClassifier(loss_function='Logloss', silent=True, random_state=42)
model.fit(train_dataset)

pred = model.predict(valid_dataset)
score = roc_auc_score(valid_y.values, pred)
print(score)

submit['Survived'] = model.predict(test_dataset)
submit.to_csv('#4.csv', index=False)

0.8194444444444445


## #5 Cabin, Ticket 제거, Age 선형회귀

In [14]:
_train = train.copy()
_test = test.copy()

age_train, age_blank = _train[~_train.Age.isnull()], _train[_train.Age.isnull()]
age_dataset = cb.Pool(age_train.drop(columns=['Survived', 'Cabin', 'Ticket']), age_train['Age'])

_, age_test = _test[~_test.Age.isnull()], _test[_test.Age.isnull()]
age_dataset2 = cb.Pool(age_test.drop(columns=['Cabin', 'Ticket']))

model = cb.CatBoostRegressor(loss_function='RMSE', silent=True, random_state=42)
model.fit(age_dataset)

pred = model.predict(age_blank.drop(columns=['Survived', 'Cabin', 'Ticket']))
_train.loc[age_blank.index]['Age'] = np.reshape(pred, (-1,))

pred = model.predict(age_dataset2)
_test.loc[age_test.index]['Age'] = np.reshape(pred, (-1,))

_train, valid = train_test_split(_train, test_size = 0.1, random_state=42)

train_x, train_y = _train.drop(columns=['Survived']), _train['Survived']
valid_x, valid_y = valid.drop(columns=['Survived']), valid['Survived']

for data in [train_x, valid_x, _test]:
    data.drop(columns=['Cabin', 'Ticket'], inplace=True)

train_dataset = cb.Pool(train_x, train_y) 
valid_dataset = cb.Pool(valid_x, valid_y)
test_dataset = cb.Pool(_test)

model = cb.CatBoostClassifier(loss_function='Logloss', silent=True, random_state=42)
model.fit(train_dataset)

pred = model.predict(valid_dataset)
score = roc_auc_score(valid_y.values, pred)
print(score)

submit['Survived'] = model.predict(test_dataset)
submit.to_csv('#5.csv', index=False)

0.8657407407407408
